In [3]:
import pandas as pd
from sklearn.datasets import load_iris
iris = load_iris()

In [5]:
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [6]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=37)

In [12]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.9666666666666667

In [13]:
from sklearn.model_selection import cross_val_score

In [14]:
cross_val_score(SVC(kernel='rbf', C=10, gamma='auto'), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [15]:
cross_val_score(SVC(kernel='rbf', C=20, gamma='auto'), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [17]:
cross_val_score(SVC(kernel='linear', C=20, gamma='auto'), iris.data, iris.target, cv=5)

array([1.        , 1.        , 0.9       , 0.93333333, 1.        ])

In [19]:
import numpy as np
kernels = ['rbf', 'linear']
C = [1, 10, 20]
avg_score = {}

for kVal in kernels:
    for cVal in C:
        cv_scores = cross_val_score(SVC(kernel=kVal, C=cVal, gamma='auto'), iris.data, iris.target, cv=5)
        avg_score[kVal + " - " + str(cVal)] = np.average(cv_scores)
        
avg_score

{'rbf - 1': 0.9800000000000001,
 'rbf - 10': 0.9800000000000001,
 'rbf - 20': 0.9666666666666668,
 'linear - 1': 0.9800000000000001,
 'linear - 10': 0.9733333333333334,
 'linear - 20': 0.9666666666666666}

In [20]:
# using grid search CV

from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(SVC(gamma='auto'), {
    'C': [1, 10, 20],
    'kernel': ['rbf', 'linear']
}, cv=5, return_train_score=False)

clf.fit(iris.data, iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00232873, 0.00340056, 0.00149016, 0.00129604, 0.00233817,
        0.00167885]),
 'std_fit_time': array([0.0003615 , 0.00079918, 0.00062348, 0.00039139, 0.00194723,
        0.00038295]),
 'mean_score_time': array([0.00166826, 0.00191102, 0.00104165, 0.000914  , 0.00189743,
        0.00076509]),
 'std_score_time': array([0.00062644, 0.00049861, 0.00063861, 0.00050626, 0.00091421,
        0.00038934]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],


In [21]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002329,0.000361,0.001668,0.000626,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.003401,0.000799,0.001911,0.000499,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001490,0.000623,0.001042,0.000639,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.001296,0.000391,0.000914,0.000506,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.002338,0.001947,0.001897,0.000914,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.001679,0.000383,0.000765,0.000389,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [23]:
df[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [24]:
dir(clf)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 '_validate_params',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 '

In [25]:
clf.best_score_

0.9800000000000001

In [26]:
clf.best_estimator_

SVC(C=1, gamma='auto')

In [27]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [28]:
# to optimize computational cost
from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(SVC(gamma='auto'), {
        'C': [1, 10, 20],
        'kernel': ['rbf', 'linear']
    }, cv=5, return_train_score=False, n_iter=2)

# will only check any 2 poss

In [31]:
rs.fit(iris.data, iris.target)
rs.cv_results_
df = pd.DataFrame(rs.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002562,0.000558,0.001737,0.000650,linear,1,"{'kernel': 'linear', 'C': 1}",0.966667,1.0,0.966667,0.966667,1.0,0.98,0.01633,1
1,0.002481,0.000626,0.001929,0.000942,rbf,10,"{'kernel': 'rbf', 'C': 10}",0.966667,1.0,0.966667,0.966667,1.0,0.98,0.01633,1


# Finding the best model for iris dataset

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [33]:
model_params = {
    'svm': {
        'model': SVC(gamma='auto'),
        'params': {
            'C': [1, 10, 20],
            'kernel': ['rbf', 'linear']
        }
    },
    'rfc': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [1, 5, 10],
        }
    },
    'lr': {
        'model': LogisticRegression(solver='liblinear', multi_class='auto'),
        'params': {
            'C': [1, 5, 10]
        }
    }
}

In [34]:
scores = []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

In [35]:
df = pd.DataFrame(scores)
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,rfc,0.960000,{'n_estimators': 5}
2,lr,0.966667,{'C': 5}


# Exercise

In [40]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.datasets import load_digits

In [41]:
digits = load_digits()

In [49]:

model_params = {
    'support_vector': {
        'model': SVC(),
        'params': {
            'C': [0.1, 1, 10, 100],
            'kernel': ['linear', 'rbf'],
            'gamma': ['scale', 'auto'],
            'probability': [True]
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['auto', 'sqrt']
        }
    },
    'decision_tree': {
        'model': RandomForestClassifier(),
        'params': {
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },
    'logistic_reg': {
        'model': LogisticRegression(),
        'params': {
            'penalty': ['l2', 'none'],  
            'C': [0.1, 1.0, 10.0],
            'solver': ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'],
            'max_iter': [100, 200, 500]
        }
    },
    'gaussian_nb': {
        'model': GaussianNB(),
        'params': {}
    },
    'multinomial_nb': {
        'model': MultinomialNB(),
        'params': {
            'alpha': [0.1, 0.5, 1.0],
            'fit_prior': [True, False]
        }
    }
}

In [50]:
%%time

model_scores = []

for model_name, mp in model_params.items():
    clf = RandomizedSearchCV(estimator=mp['model'], param_distributions=mp['params'], cv=5, n_iter=5)
    clf.fit(digits.data, digits.target)
    model_scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

C:\Users\PIYUSH_M\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
10 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\PIYUSH_M\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\PIYUSH_M\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\Users\PIYUSH_M\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\PIYUSH_M\anaconda3\Lib

In [52]:
df = pd.DataFrame(model_scores)
df

CPU times: total: 0 ns
Wall time: 1.56 ms


,model,best_score,best_params
0,support_vector,0.947697,"{'probability': True, 'kernel': 'linear', 'gam..."
1,random_forest,0.938245,"{'n_estimators': 200, 'min_samples_split': 2, ..."
2,decision_tree,0.936024,"{'min_samples_split': 10, 'min_samples_leaf': ..."
3,logistic_reg,0.922114,"{'solver': 'liblinear', 'penalty': 'l2', 'max_..."
4,gaussian_nb,0.806928,{}
5,multinomial_nb,0.870907,"{'fit_prior': True, 'alpha': 0.1}"
